In [119]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict, Counter

from sklearn.ensemble import RandomForestRegressor

In [87]:
df_agri = pd.read_csv('/private/tmp/test_agged.csv')
# get uniq lat longs
uniq_latlngs = set(tuple(t) for t in df_agri[['lat', 'lng']].values)
lat_tick = np.median(sorted(np.diff(df_agri['lat'].unique())))
lng_tick = np.median(sorted(np.diff(df_agri['lng'].unique())))

In [124]:
weather_aggs = defaultdict(Counter)
weather_vals = ['prcp', 'tmin', 'tmax', 'vp']
for val in weather_vals:
    df_weather = pd.read_csv('/private/tmp/{}.csv'.format(val), index_col=0)
    for lat, lng in uniq_latlngs:
        lat_mask = (df_weather['lat']<(lat+lat_tick)) & (df_weather['lat']>=(lat))
        lng_mask = (df_weather['lng']>=(lng+lng_tick)) & (df_weather['lng']<(lng))
        weather_aggs[(lat, lng)][val] += df_weather[lat_mask&lng_mask]['val'].mean()

In [146]:
df_weather_by_loc = pd.DataFrame(weather_aggs).T
df_weather_by_loc.index = df_weather_by_loc.index.set_names(['lat', 'lng'])
df_weather_by_loc = df_weather_by_loc.dropna()

In [147]:
# corn only
df_corn = df_agri[df_agri['type']==1].set_index(['lat', 'lng'])

In [150]:
df_plot = df_corn.join(df_weather_by_loc, how='inner')

In [151]:
rfr = RandomForestRegressor()
rfr.fit(df_plot[weather_vals], df_plot['ct'])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [152]:
ct_ = rfr.predict(df_weather_by_loc[weather_vals])

In [154]:
df_weather_by_loc['pred_corn'] = ct_

In [164]:
df_out = df_weather_by_loc[['pred_corn']].reset_index()
df_out['type']=1

In [166]:
df_out.to_csv('/private/tmp/test_agged_predictions.csv')